Módulo de análisis gramátical que hace parte de SEÑAL: Programa para el análisis lingüístico y evaluación de composiciones de español.

© Restrepo-Ramos
**2022**

# Módulo de compilación gramatical

Este módulo permite medir el tipo de verbos conjugados en tus escrituras. Primero, el programa divide en unidades sintácticas tus oraciones utilizando un modelo pre-entrenado de español (más info [aquí](https://universaldependencies.org/)). El programa utiliza este modelo como muestra para analizar tus composiciones. 

Por último, el programa revisa si estás utilizando el vocabulario de los capítulos.


### Requisitos
No hay requisitos técnicos. Sólo presta atención a la presentación de tu instructor.

### Materiales
1. Módelo pre-entrenado de español ***es.udpipe***
2. Tu ***muestra.txt***

### Instrucciones
1. Ejecuta todas las líneas de código leyendo la descripción de cada funcionalidad del programa
2. interpreta los resultados de acuerdo a las distintas medidas gramaticales


**Paso 1**. Instalamos UDPipe

UDPipe es el módulo que nos va a permitir dividir cada oración de tus composiciones en unidades sintácticas (i.e., *parsing*). El programa nos va a arrojar un archivo en formato ***.conllu*** 

In [ ]:
!pip install ufal.udpipe

**Paso 2**

Ejecutamos las siguiente línea de código. Aunque parece complicado, está parte del programa crea *definiciones* para que cuando subas tu composición, el programa pueda realizar la división sintáctica.

In [ ]:
import sys
import ufal.udpipe

class Model:
    def __init__(self, path):
        """Load given model."""
        self.model = ufal.udpipe.Model.load(path)
        if not self.model:
            raise Exception("Cannot load UDPipe model from file '%s'" % path)

    def tokenize(self, text):
        """Tokenize the text and return list of ufal.udpipe.Sentence-s."""
        tokenizer = self.model.newTokenizer(self.model.DEFAULT)
        if not tokenizer:
            raise Exception("The model does not have a tokenizer")
        return self._read(text, tokenizer)

    def read(self, text, in_format):
        """Load text in the given format (conllu|horizontal|vertical) and return list of ufal.udpipe.Sentence-s."""
        input_format = ufal.udpipe.InputFormat.newInputFormat(in_format)
        if not input_format:
            raise Exception("Cannot create input format '%s'" % in_format)
        return self._read(text, input_format)

    def _read(self, text, input_format):
        input_format.setText(text)
        error = ufal.udpipe.ProcessingError()
        sentences = []

        sentence = ufal.udpipe.Sentence()
        while input_format.nextSentence(sentence, error):
            sentences.append(sentence)
            sentence = ufal.udpipe.Sentence()
        if error.occurred():
            raise Exception(error.message)

        return sentences

    def tag(self, sentence):
        """Tag the given ufal.udpipe.Sentence (inplace)."""
        self.model.tag(sentence, self.model.DEFAULT)

    def parse(self, sentence):
        """Parse the given ufal.udpipe.Sentence (inplace)."""
        self.model.parse(sentence, self.model.DEFAULT)

    def write(self, sentences, out_format):
        """Write given ufal.udpipe.Sentence-s in the required format (conllu|horizontal|vertical)."""

        output_format = ufal.udpipe.OutputFormat.newOutputFormat(out_format)
        output = ''
        for sentence in sentences:
            output += output_format.writeSentence(sentence)
        output += output_format.finishDocument()

        return output


**Paso 3**

Subimos el módelo preentrenado de español



In [ ]:
model = Model('es.udpipe')

**Paso 4**

Subimos nuestro ensayo a la memoria. Si tienes problemas con MAC cambia el encoding a: ISO-8859-1

In [ ]:
ifile = open("muestra.txt", mode='r', encoding='utf-8')
texto = ifile.read()
ifile.close()

**Paso 5**

Utilizamos el modelo preentrenado y realizamos la división sintáctica de nuestras composiciones (*Parsing*).

In [ ]:
sentences = model.tokenize(texto)
for s in sentences:
    model.tag(s)
    model.parse(s)
conllu = model.write(sentences, "conllu")

**Paso 6**

Ahora vamos a examinar los tipos de tiempos verbales que estás utilizando en tus composiciones.

Explicaré todo esto en clase.



In [ ]:
import sys
import re

words = []
nouns = []
verbs = []
verbs_pres_ind = []
verbs_pres_sub = []
verbs_pret = []
verbs_imp_ind = []
verbs_imp_sub = []
verbs_pres_sub = []
verbs_fut = []
verbs_cond = []
adj = []
adv = []


for sent in conllu.split('\n\n'):
    sent_id = '' # the sentence id
    verbs_list = 0
    verbs_pres_list = 0
    verbs_pres_sub_list = 0
    verbs_imp_list = 0
    verbs_imp_sub_list = 0
    verbs_pres_subj_list = 0
    verbs_pret_list = 0
    verbs_fut_list = 0
    verbs_cond_list = 0
    nouns_list = 0
    adj_list = 0
    adv_list = 0
    others = 0
    word_list = 0

    if sent.strip() == '': # if there is no data in the sentence then skip it
        continue
    #for each of the lines in the sentence
    for line in sent.split('\n'):
        if line[0] !='#':
            word_list += 1

        if (re.findall('Tense=Pres', line)) and (re.findall('Mood=Sub', line)):
            verbs_list += 1
            verbs_pres_sub_list += 1

        if (re.findall('Tense=Pres', line)) and (re.findall('Mood=Ind', line)):
            verbs_list += 1
            verbs_pres_list += 1

        if line.count('Tense=Past'):
            verbs_list += 1
            verbs_pret_list += 1

        if (re.findall('Tense=Imp', line)) and (re.findall('Mood=Ind', line)):
            verbs_list += 1
            verbs_imp_list += 1

        if (re.findall('Tense=Imp', line)) and (re.findall('Mood=Sub', line)):
            verbs_list += 1
            verbs_imp_sub_list += 1

        if line.count('Tense=Fut'):
            verbs_list += 1
            verbs_fut_list += 1

        if line.count('Mood=Cnd'):
            verbs_list += 1
            verbs_cond_list += 1

        if line.count('NOUN'):
            nouns_list += 1

        if line.count('ADJ'):
            adj_list += 1

        if line.count('ADV'):
            adv_list += 1

        if (re.findall('Tense=Pres', line)) and (re.findall('Mood=Sub', line)):
          verbs_list += 1
          verbs_pres_sub_list += 1
                
    for x in range((nouns_list)):
        nouns.append(nouns_list)
    for x in range((verbs_list)):
        verbs.append(verbs_list)
    for x in range((verbs_pres_list)):
        verbs_pres_ind.append(verbs_pres_list)
    for x in range((verbs_pres_sub_list)):
        verbs_pres_sub.append(verbs_pres_sub_list)
    for x in range((verbs_imp_list)):
        verbs_imp_ind.append(verbs_imp_list)
    for x in range((verbs_imp_sub_list)):
        verbs_imp_sub.append(verbs_imp_sub_list)
    for x in range((verbs_pres_sub_list)):
        verbs_pres_sub.append(verbs_pres_sub_list)
    for x in range((verbs_pret_list)):
        verbs_pret.append(verbs_pret_list)
    for x in range((verbs_fut_list)):
        verbs_fut.append(verbs_fut_list)
    for x in range((verbs_cond_list)):
        verbs_cond.append(verbs_cond_list)
    for x in range((adj_list)):
        adj.append(adj_list)
    for x in range((adv_list)):
        adv.append(adv_list)
    for x in range((word_list)):
        words.append(word_list)


print("---------------------------------------------------------------")
print("Total de palabras en el texto: ", len(words))
print("De las cuales ", len(nouns), "/", len(words), "son sustantivos")
print("De las cuales ", len(adj), "/", len(words), "son adjetivos")
print("De las cuales ", len(adv), "/", len(words), "son adverbios")
print("De las cuales ", len(verbs), "/", len(words), "son verbos")
print("\tDe estos verbos: ")
print("\t\t", len(verbs_pres_ind), "/", len(verbs), " están en PRESENTE DE INDICATIVO")
print("\t\t", len(verbs_pres_sub), "/", len(verbs), " están en PRESENTE DE SUBJUNTIVO")
print("\t\t", len(verbs_imp_ind), "/", len(verbs), " están en IMPERFECTO DE INDICATIVO")
print("\t\t", len(verbs_imp_sub), "/", len(verbs), " están en IMPERFECTO DE SUBJUNTIVO")
print("\t\t", len(verbs_pret), "/", len(verbs), " están en PRETÉRITO DE INDICATIVO")
print("\t\t", len(verbs_fut), "/", len(verbs), " están en FUTURO SIMPLE")
print("\t\t", len(verbs_cond), "/", len(verbs), " están en CONDICIONAL SIMPLE")
print("---------------------------------------------------------------")



==============================AHORA TÚ=========================
Compara los tiempos verbales de tu ensayo 2. ¿Qué diferencias y similitudes encontraste? ¿En qué tiempo verbal debes enfocarte más?